## 1️⃣ Check GPU

In [ ]:
# Verify GPU availability
!nvidia-smi

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2️⃣ Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create project directory in Drive
import os
project_dir = '/content/drive/MyDrive/Face_3D_Reconstruction'
os.makedirs(project_dir, exist_ok=True)
print(f"✅ Project directory: {project_dir}")

## 3️⃣ Clone Repository

In [ ]:
# Clone your GitHub repo
%cd /content
!git clone https://github.com/Rohith-Krishna2612/Face_3D_Reconstruction.git
%cd Face_3D_Reconstruction

!ls -la

## 4️⃣ Install Dependencies

In [ ]:
# Install PyTorch with CUDA (Colab comes with it, but ensure correct version)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q

# Install other dependencies
!pip install -r requirements.txt -q

print("\n✅ Dependencies installed!")

## 5️⃣ Setup FFHQ Dataset

**Option A: Download from Kaggle** (Recommended - one-time setup)

In [ ]:
# Setup Kaggle API
# 1. Go to https://www.kaggle.com/settings/account
# 2. Click "Create New API Token" to download kaggle.json
# 3. Upload kaggle.json using the file upload button below

from google.colab import files
print("📤 Upload your kaggle.json file:")
uploaded = files.upload()

# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("\n✅ Kaggle API configured!")

In [ ]:
# Download FFHQ dataset from Kaggle (only need to do once)
import os

# Check if dataset already exists in Drive
drive_dataset_path = '/content/drive/MyDrive/Face_3D_Reconstruction/data/ffhq'

if os.path.exists(drive_dataset_path) and len(os.listdir(drive_dataset_path)) > 1000:
    print(f"✅ Dataset already exists in Drive: {drive_dataset_path}")
    print(f"✅ Image count: {len(os.listdir(drive_dataset_path))}")
else:
    print("⬇️ Downloading FFHQ dataset from Kaggle...")
    print("⚠️ This will take 10-15 minutes (13GB download)\n")
    
    # Download using kagglehub
    !pip install kagglehub -q
    
    import kagglehub
    path = kagglehub.dataset_download("arnaud58/flickrfaceshq-dataset-ffhq")
    
    print(f"\n✅ Downloaded to: {path}")
    
    # Move to Drive for persistence
    print("\n📁 Moving to Google Drive for persistence...")
    !mkdir -p {drive_dataset_path}
    !cp -r {path}/* {drive_dataset_path}/
    
    print(f"\n✅ Dataset saved to Drive: {drive_dataset_path}")
    print(f"✅ You won't need to download again!")

**Option B: Upload from Local** (If you already downloaded)

In [ ]:
# Skip this if you used Option A above
# Manually upload FFHQ dataset to Google Drive at:
# /content/drive/MyDrive/Face_3D_Reconstruction/data/ffhq/

# Verify dataset
import os
drive_dataset_path = '/content/drive/MyDrive/Face_3D_Reconstruction/data/ffhq'
if os.path.exists(drive_dataset_path):
    count = len([f for f in os.listdir(drive_dataset_path) if f.endswith('.png')])
    print(f"✅ Found {count} images in Drive")
else:
    print("❌ Dataset not found. Please use Option A or upload manually.")

## 6️⃣ Create Symlink to Dataset

In [ ]:
# Create symlink from Drive to local data folder (faster access)
import os

local_data_dir = '/content/Face_3D_Reconstruction/data'
drive_dataset_path = '/content/drive/MyDrive/Face_3D_Reconstruction/data/ffhq'

os.makedirs(local_data_dir, exist_ok=True)

# Remove existing symlink if present
if os.path.exists(f"{local_data_dir}/ffhq"):
    !rm -rf {local_data_dir}/ffhq

# Create symlink
!ln -s {drive_dataset_path} {local_data_dir}/ffhq

# Verify
count = len([f for f in os.listdir(f"{local_data_dir}/ffhq") if f.endswith('.png')])
print(f"✅ Symlink created: {local_data_dir}/ffhq")
print(f"✅ Accessible images: {count}")

## 7️⃣ Update Config for Colab (512×512)

In [ ]:
# Update config.yaml for Colab T4 GPU
import yaml

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Optimize for T4 GPU (16GB VRAM)
config['dataset']['resolution'] = 512  # Higher resolution
config['training']['batch_size'] = 4  # Bigger batch
config['training']['num_workers'] = 2
config['training']['gradient_accumulation_steps'] = 2  # Less needed with batch=4

# Save checkpoints to Drive
config['paths']['checkpoints'] = '/content/drive/MyDrive/Face_3D_Reconstruction/checkpoints'
config['paths']['logs'] = '/content/drive/MyDrive/Face_3D_Reconstruction/logs'
config['paths']['output'] = '/content/drive/MyDrive/Face_3D_Reconstruction/output'

# Save updated config
with open('config.yaml', 'w') as f:
    yaml.dump(config, f)

print("✅ Config updated for Colab T4:")
print(f"   Resolution: {config['dataset']['resolution']}×{config['dataset']['resolution']}")
print(f"   Batch size: {config['training']['batch_size']}")
print(f"   Gradient accumulation: {config['training']['gradient_accumulation_steps']}")
print(f"   Effective batch: {config['training']['batch_size'] * config['training']['gradient_accumulation_steps']}")

## 8️⃣ Quick Test (Optional - 15 minutes)

In [ ]:
# Quick test with 1000 samples to verify everything works
!python quick_train.py --epochs 2 --max-samples 1000

print("\n✅ Quick test complete! If this worked, proceed to full training.")

## 9️⃣ Start Full Training (512×512)

**⚠️ Important Notes:**
- Training will take ~5-7 hours
- Colab Free has 12-hour limit (should be enough)
- Don't close browser tab
- Checkpoints auto-save to Google Drive every 5 epochs

In [ ]:
# Option 1: Train with 5000 samples (recommended - ~5 hours)
!python quick_train.py --epochs 20 --max-samples 5000

# Option 2: Full dataset training (if you have Colab Pro - ~12 hours)
# !python train.py

print("\n🎉 Training complete!")
print("✅ Best model saved to Google Drive")

## 🔟 Monitor Training (Run in parallel)

In [ ]:
# Start TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/Face_3D_Reconstruction/logs/codeformer

## 1️⃣1️⃣ Check Sample Outputs

In [ ]:
# View sample restoration outputs
import matplotlib.pyplot as plt
from PIL import Image
import os

output_dir = '/content/drive/MyDrive/Face_3D_Reconstruction/output/samples'
samples = sorted(os.listdir(output_dir))[-5:]  # Last 5 samples

fig, axes = plt.subplots(1, 5, figsize=(20, 4))
for idx, sample in enumerate(samples):
    img = Image.open(os.path.join(output_dir, sample))
    axes[idx].imshow(img)
    axes[idx].axis('off')
    axes[idx].set_title(sample)
plt.tight_layout()
plt.show()

print(f"\n✅ Showing last 5 sample outputs from: {output_dir}")

## 1️⃣2️⃣ Download Best Model

In [ ]:
# Download the best trained model to your local machine
from google.colab import files

checkpoint_dir = '/content/drive/MyDrive/Face_3D_Reconstruction/checkpoints/codeformer'
best_model = os.path.join(checkpoint_dir, 'best_checkpoint.pth')

if os.path.exists(best_model):
    print(f"📥 Downloading best model: {best_model}")
    files.download(best_model)
    print("\n✅ Download complete!")
    print("\n📂 Place it in your local: checkpoints/codeformer/")
else:
    print(f"❌ Best model not found at: {best_model}")
    print("\nAvailable checkpoints:")
    !ls -lh {checkpoint_dir}

## 1️⃣3️⃣ Training Metrics

In [ ]:
# Display final training metrics
import json

metrics_file = '/content/drive/MyDrive/Face_3D_Reconstruction/logs/codeformer/metrics.json'

if os.path.exists(metrics_file):
    with open(metrics_file, 'r') as f:
        metrics = json.load(f)
    
    print("📊 Final Training Metrics:")
    print("=" * 50)
    print(f"Best Epoch: {metrics.get('best_epoch', 'N/A')}")
    print(f"Best PSNR: {metrics.get('best_psnr', 'N/A'):.2f} dB")
    print(f"Best SSIM: {metrics.get('best_ssim', 'N/A'):.4f}")
    print(f"Final Loss: {metrics.get('final_loss', 'N/A'):.4f}")
    print(f"Training Time: {metrics.get('total_time', 'N/A')}")
    print("=" * 50)
else:
    print("⚠️ Metrics file not found. Check logs directory.")

## 📝 Summary

### What You Just Did:
✅ Trained CodeFormer at **512×512** resolution (better than local 256×256)  
✅ Used T4 GPU with 16GB VRAM (4x more than RTX 3050)  
✅ Batch size 4 (2x larger than local)  
✅ Training time: ~5-7 hours (similar to local but better quality)  
✅ All outputs saved to Google Drive  

### Next Steps:
1. Download `best_checkpoint.pth` from cell above
2. Place in your local: `checkpoints/codeformer/`
3. Test with web interface: `cd backend && python main.py`
4. Deploy Phase 1 complete! 🎉

### Files in Your Google Drive:
```
/MyDrive/Face_3D_Reconstruction/
├── checkpoints/codeformer/
│   ├── best_checkpoint.pth (download this!)
│   ├── epoch_5.pth
│   ├── epoch_10.pth
│   └── ...
├── logs/codeformer/
│   └── (TensorBoard logs)
├── output/samples/
│   └── (sample images)
└── data/ffhq/
    └── (70K images - keep for future use)
```